In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnull, lit, regexp_extract

# =====================
# Inicializar Spark
# =====================
spark = SparkSession.builder \
    .appName("ManejoErroresYValidacionEmail") \
    .getOrCreate()

# =====================
# 1. Crear datos artificiales con nulls y duplicados
# =====================
data = [
    (1, "Carlos", "carlos@example.com", 5000),
    (2, "Ana", None, 6000),
    (3, "Luis", "luis@example.com", None),
    (4, None, "julia@example.com", 7000),
    (5, "Carlos", "carlos@example.com", 5000),  # Duplicado
    (6, "Pedro", "correo_invalido", 4500),
    (7, None, None, None)
]

columns = ["id", "nombre", "email", "salario"]

df = spark.createDataFrame(data, columns)

print("📋 Datos originales:")
df.show(truncate=False)

# =====================
# 2. Filtrar filas con valores nulos usando dropna
# =====================
print("🧹 dropna() - Eliminar filas con cualquier valor null:")
df_dropna = df.dropna()
df_dropna.show(truncate=False)

# =====================
# 3. Rellenar valores nulos con fillna
# =====================
print("🔧 fillna() - Rellenar nulos con valores por defecto:")
df_fillna = df.fillna({
    "nombre": "Desconocido",
    "email": "sin_email@example.com",
    "salario": 0
})
df_fillna.show(truncate=False)

# =====================
# 4. Identificar registros con campos nulos usando isNull
# =====================
print("🔍 Filtrar donde salario IS NULL:")
df_salario_null = df.filter(col("salario").isNull())
df_salario_null.show(truncate=False)

# =====================
# 5. Eliminar registros duplicados
# =====================
print("🧼 dropDuplicates() - Eliminar filas duplicadas por email:")
df_deduplicado = df.dropDuplicates(subset=["email"])
df_deduplicado.show(truncate=False)

# =====================
# 6. Validación de correos con expresión regular
# =====================
# Expresión regular básica para validar emails
email_regex = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'

df_validacion = df.withColumn(
    "email_valido",
    regexp_extract(col("email"), email_regex, 0) != ""
)

print("✅ Validación de correos electrónicos:")
df_validacion.select("email", "email_valido").show(truncate=False)

# 🧹 Ejercicio – Limpieza de Datos de Contacto con PySpark

## 🎯 Objetivo
Aplicar técnicas de limpieza y validación de datos en PySpark, utilizando un conjunto de datos generado con Faker que contiene información de contacto con errores comunes como valores nulos, espacios en blanco, duplicados y formatos inválidos.

---

## 📁 Dataset

Utiliza la tabla default.cliente

El conjunto de datos tiene la siguiente estructura:

| nombre       | email                | telefono        | direccion                |
|--------------|----------------------|------------------|---------------------------|
| Carlos Ruiz  | carlos@gmail.com     | 123-456-7890     | Calle Falsa 123           |
| Ana Torres   |                     | 555-123-4567     | Avenida Siempre Viva 742 |
| Pedro Gómez  | None                 |                  | None                      |
| Carlos Ruiz  | carlos@gmail.com     | 123-456-7890     | Calle Falsa 123           |

⚠️ Este dataset contiene:
- Nulos en `email` y `direccion`
- Strings vacíos en `telefono`
- Registros duplicados
- Correos potencialmente mal formateados

---

## 🛠 Actividades

### 1️⃣ Eliminar registros con campos incompletos

Usar `dropna()` para eliminar filas que tengan valores nulos o vacíos en cualquier campo.

---

### 2️⃣ Identificar registros con campos nulos

Filtrar los registros donde alguna columna específica (por ejemplo, `email`) esté vacía o nula.

---

### 3️⃣ Rellenar valores faltantes

Usar `fillna()` para reemplazar los `nulls` con valores por defecto.

---

### 4️⃣ Eliminar registros duplicados

Eliminar filas duplicadas completas o basadas en columnas clave como `email`.

---

### 5️⃣ Validar formato de emails con regexp

Utilizar una expresión regular para detectar correos mal formateados.

---

## 📂 Entrega

Subí un notebook `.ipynb` o un script `.py` que incluya:
- Cada paso del proceso de limpieza.
- Muestras de los datos antes y después (`.show()`).
- Comentarios explicando cada transformación.

---

## ✅ Criterios de evaluación

- ✔ Uso correcto de `dropna`, `isNull`, `fillna`, `dropDuplicates` y `regexp_extract`.
- ✔ Código ordenado, funcional y comentado.
- ✔ Resultados interpretables y consistentes.

---

💡 *Este ejercicio te prepara para trabajar con datos reales, que muchas veces vienen incompletos, sucios o mal estructurados. Aprender a limpiarlos de forma eficiente es clave para cualquier pipeline de datos.*
